In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: umraimlnd
Azure region: centralus
Subscription id: b1d4fcba-3863-40a9-84b1-4b0f82fd16cc
Resource group: rg-umrdataexternalization-dev


In [2]:
from azureml.core import Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
CLUSTER_NAME = "AIMLND"
# retrieve existing cluster or create a new one
try:
    compute_target = ComputeTarget(workspace=ws, name=CLUSTER_NAME)
    print("Found existing cluster")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_DS12_V2", max_nodes=4
    )
    compute_target = ComputeTarget.create(ws, CLUSTER_NAME, compute_config)
compute_target.wait_for_completion(show_output=True)

Found existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [15]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig, Workspace
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

import os
subscription_id = ws.subscription_id,
resource_group = ws.resource_group,
SOURCE_FOLDER = "/Users/sgummad9/training"
#credential = DefaultAzureCredential()
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, ws)
# Specify parameter sampler
ps = RandomParameterSampling( {
'--batch_size' : choice(0.001,0.01,0.1,1,10),
'--frequency': choice(100,200,300),
})
# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.12)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

2022-12-09:06:30:38,990 INFO     [environment.py:98] No environment configuration found.
2022-12-09:06:30:38,991 INFO     [managed_identity.py:46] ManagedIdentityCredential will use App Service managed identity
2022-12-09:06:30:38,995 INFO     [environment.py:98] No environment configuration found.
2022-12-09:06:30:38,995 INFO     [managed_identity.py:46] ManagedIdentityCredential will use App Service managed identity


In [4]:
# Create a ScriptRunConfig Object to specify the configuration details of your training job

src = ScriptRunConfig(source_directory= '.',
                      command=['python','train.py'],
                      compute_target=compute_target,
                      environment=sklearn_env)


# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.


In [5]:
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     run_config=src,
                                     max_total_runs=10,
                                     max_concurrent_runs=4)
print ('Hyperdrive config set')

Hyperdrive config set


In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show
hyperdrive_run.wait_for_completion(show_output=True)


RunId: HD_76df07e8-d226-425a-a853-f18faf80b867
Web View: https://ml.azure.com/runs/HD_76df07e8-d226-425a-a853-f18faf80b867?wsid=/subscriptions/b1d4fcba-3863-40a9-84b1-4b0f82fd16cc/resourcegroups/rg-umrdataexternalization-dev/workspaces/umraimlnd&tid=db05faca-c82a-4b9d-b9c5-0f64b6755421

Streaming azureml-logs/hyperdrive.txt

[2022-12-09T05:55:07.968560][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-12-09T05:55:08.5208105Z][SCHEDULER][INFO]Scheduling job, id='HD_76df07e8-d226-425a-a853-f18faf80b867_0' 
[2022-12-09T05:55:08.6404223Z][SCHEDULER][INFO]Scheduling job, id='HD_76df07e8-d226-425a-a853-f18faf80b867_1' 
[2022-12-09T05:55:08.7718159Z][SCHEDULER][INFO]Scheduling job, id='HD_76df07e8-d226-425a-a853-f18faf80b867_2' 
[2022-12-09T05:55:08.787836][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-12-09T05:55:08.8563566Z][SCHEDULER][INFO]Scheduling job, id='HD_76df07e8-d226-425a-a853-f18faf80b867_3' 

{'runId': 'HD_76df07e8-d226-425a-a853-f18faf80b867',
 'target': 'AIMLND',
 'status': 'Completed',
 'startTimeUtc': '2022-12-09T05:55:07.181411Z',
 'endTimeUtc': '2022-12-09T06:02:08.431554Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '08014288-b42c-41b3-a28d-d64635710d64',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1022-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': '15',
  'score': '0.9118361153262519',
  'best_child_run_id': 'HD_76df07e8-d226-425a-a853-f18faf80b867_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_76df07e8-d226-425a-a853-f18faf80b867_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType':

In [7]:
import joblib
# Get your best run and save the model from that run.

print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

best_run = hyperdrive_run.get_best_run_by_primary_metric()

print("Best run metrics :",best_run.get_metrics())

print("Best run details :",best_run.get_details())

print("Best run file names :",best_run.get_file_names())


[{'run_id': 'HD_76df07e8-d226-425a-a853-f18faf80b867_8', 'hyperparameters': None, 'best_primary_metric': 0.9118361153262519, 'status': 'Completed'}, {'run_id': 'HD_76df07e8-d226-425a-a853-f18faf80b867_9', 'hyperparameters': None, 'best_primary_metric': 0.9118361153262519, 'status': 'Completed'}, {'run_id': 'HD_76df07e8-d226-425a-a853-f18faf80b867_7', 'hyperparameters': None, 'best_primary_metric': 0.9118361153262519, 'status': 'Completed'}, {'run_id': 'HD_76df07e8-d226-425a-a853-f18faf80b867_6', 'hyperparameters': None, 'best_primary_metric': 0.9118361153262519, 'status': 'Completed'}, {'run_id': 'HD_76df07e8-d226-425a-a853-f18faf80b867_5', 'hyperparameters': None, 'best_primary_metric': 0.9118361153262519, 'status': 'Completed'}, {'run_id': 'HD_76df07e8-d226-425a-a853-f18faf80b867_4', 'hyperparameters': None, 'best_primary_metric': 0.9118361153262519, 'status': 'Completed'}, {'run_id': 'HD_76df07e8-d226-425a-a853-f18faf80b867_1', 'hyperparameters': None, 'best_primary_metric': 0.91183

In [1]:
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.data.dataset_factory import TabularDatasetFactory
ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])
ds.to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32946,37,management,married,university.degree,no,no,yes,cellular,jul,fri,...,7,999,0,nonexistent,1.4,93.918,-42.7,4.957,5228.1,no
32947,26,admin.,single,university.degree,no,no,no,cellular,may,tue,...,4,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
32948,31,blue-collar,single,basic.9y,no,no,no,cellular,apr,mon,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no


In [2]:
from train import clean_data
# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [14]:
x

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_cellular,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown
0,57,1,0,0,1,5,1,371,1,999,...,1,0,0,0,0,1,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,0,1,0,0,0,0,0,0,0,1
2,33,1,0,0,0,5,5,52,1,999,...,1,0,0,0,1,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,0,1,0,0,0,1,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,1,0,0,1,7,1,116,1,999,...,1,0,1,0,0,0,0,0,0,0
32946,37,1,0,0,1,7,5,69,7,999,...,1,0,0,0,0,0,0,0,1,0
32947,26,0,0,0,0,5,2,135,4,999,...,1,0,0,0,0,0,0,0,1,0
32948,31,0,0,0,0,4,1,386,1,999,...,1,0,0,0,1,0,0,0,0,0


In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    task='classification',
    iterations=30,
    label_column_name='y',
    iteration_timeout_minutes=5,
    primary_metric='accuracy',
    training_data=ds,
    n_cross_validations=2)

In [9]:
# Submit your automl run
#from azureml.core.experiment import Experiment
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()
automl_experiment = Experiment(ws,'automl_experiment')
automl_run = automl_experiment.submit(config=automl_config)
automl_run.wait_for_completion(show_output=True)

2022-12-09:16:09:00,937 INFO     [workspace.py:291] Found the config file in: /config.json
2022-12-09:16:09:02,395 WARNING  [local_experiment_driver.py:147] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
automl_experiment,AutoML_4882f58b-3f89-4c53-a798-0342f82dfcbd,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2022/12/09 16:11:15 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2022-12-09:16:33:31,527 INFO     [explanation_client.py:334] Using default datastore for uploads


Experiment,Id,Type,Status,Details Page,Docs Page
automl_experiment,AutoML_4882f58b-3f89-4c53-a798-0342f82dfcbd,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |yes                             |32950                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_4882f58b-3f89-4c53-a798-0342f82dfcbd',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-12-09T16:09:07.167265Z',
 'endTimeUtc': '2022-12-09T16:32:06.588821Z',
 'services': {},
 'properties': {'num_iterations': '30',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_experiment","subscription_id":"b1d4fcba-3863-40a9-84b1-4b0f82fd16cc","resource_group":"rg-umrdataexternalization-dev","workspace_name":"umraimlnd","region":"centralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_parallel_run":false,"num_procs":null,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":

# Retrieve and save your best automl model.
best_run, fitted_model =remote_run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

ml_client.compute.begin_delete(CLUSTER_NAME).wait()
